# ETAPA 01 - Configuração do ambiente

In [1]:
# BLOCO 01 - IMPORTAÇÃO BASE PARA O CUIDAR.AI MULTIAGENTE

# --- Bibliotecas Padrão ---
import importlib.util
import subprocess
import sys
import os

# --- Suporte a Agentes Multiagente (será usado mais adiante) ---
import warnings  # Para suprimir alertas desnecessários

# --- Organização futura: estes serão usados nos agentes ---
# OBS: só efetivamente usados quando definirmos os agentes

# from google.adk.agents import Agent
# from google.adk.runners import Runner
# from google.adk.sessions import InMemorySessionService
# from google.adk.tools import google_search
# from google.genai import types


In [2]:
# BLOCO 02 - Instalação de Bibliotecas via pip (Atualizado para Multiagentes)

# 📦 Lista de bibliotecas Python a serem verificadas/instaladas
bibliotecas = {
    "google.generativeai": "google-generativeai",  # SDK v1.5 (legado)
    "google.genai": "google-genai",                # SDK v2 (Gemini + ADK)
    "gtts": "gTTS",                                 # Síntese de voz online
    "pydub": "pydub",                               # Manipulação de áudio
    "tqdm": "tqdm",                                 # Barra de progresso
    "ipywidgets": "ipywidgets",                     # Interface gráfica
    "pyttsx3": "pyttsx3",                           # Voz offline
    "google.adk": "google-adk"                      # Agentes multiagentes
}

# 🔍 Verifica e instala automaticamente as bibliotecas necessárias
for modulo, nome in bibliotecas.items():
    if importlib.util.find_spec(modulo):
        print(f"✅ {nome} já está instalado.")
    else:
        print(f"📦 Instalando {nome}...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", nome])
            if importlib.util.find_spec(modulo):
                print(f"✅ {nome} instalado com sucesso!")
            else:
                print(f"❌ {nome} ({modulo}) ainda não foi detectado após instalação.")
        except subprocess.CalledProcessError:
            print(f"❌ Erro ao tentar instalar {nome} ({modulo}). Verifique a conexão ou permissões.")


✅ google-generativeai já está instalado.
✅ google-genai já está instalado.
✅ gTTS já está instalado.
✅ pydub já está instalado.
✅ tqdm já está instalado.
✅ ipywidgets já está instalado.
✅ pyttsx3 já está instalado.
✅ google-adk já está instalado.


In [3]:
# BLOCO 03 - Verificação de Pacotes de Sistema (Somente Fora do Colab)

# 🛠️ Função para verificar se um pacote de sistema está instalado (dpkg - Debian/Ubuntu)
def is_package_installed(package):
    try:
        result = subprocess.run(['dpkg', '-l', package], capture_output=True, text=True)
        return result.returncode == 0 and package in result.stdout
    except subprocess.SubprocessError:
        return False

# 🩺 Diagnóstico interativo para verificar versões de pacotes essenciais
def diagnostico_pacotes_sistema():
    print("\n🔍 Diagnóstico: Verificando pacotes do sistema no ambiente atual...\n")
    comandos = {
        "ffmpeg": "ffmpeg -version",  # Para manipulação de áudio
        "espeak": "espeak --version"  # Para voz offline (pyttsx3 fallback)
    }
    for nome, comando in comandos.items():
        print(f"📦 {nome}: ", end="")
        try:
            resultado = subprocess.run(comando.split(), capture_output=True, text=True)
            if resultado.returncode == 0:
                print("✅ Disponível")
                print(f"🔹 Versão: {resultado.stdout.splitlines()[0]}")
            else:
                print("❌ Não encontrado ou erro ao executar.")
        except FileNotFoundError:
            print("❌ Não encontrado no ambiente.")
        except Exception as e:
            print(f"⚠️ Erro inesperado: {e}")
    print("\n🧪 Diagnóstico concluído.\n")

# 🌐 Detecta se está rodando no Google Colab
is_colab = 'google.colab' in sys.modules
system_packages = ['ffmpeg', 'espeak']

# 📦 Verifica e instala (somente em ambiente local Linux)
if not is_colab:
    print("🛠️ Verificando pacotes no ambiente local...\n")
    for package in system_packages:
        if is_package_installed(package):
            print(f"✅ {package} já está instalado.")
        else:
            print(f"📦 Instalando {package}...")
            subprocess.check_call(['apt-get', 'install', '-y', package], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            if is_package_installed(package):
                print(f"✅ {package} instalado com sucesso!")
            else:
                print(f"❌ Falha ao instalar {package}. Verifique a conexão ou permissões.")
else:
    print("ℹ️ No Colab, pacotes do sistema (ffmpeg, espeak) não são instalados diretamente.")
    diagnostico_pacotes_sistema()


ℹ️ No Colab, pacotes do sistema (ffmpeg, espeak) não são instalados diretamente.

🔍 Diagnóstico: Verificando pacotes do sistema no ambiente atual...

📦 ffmpeg: ✅ Disponível
🔹 Versão: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
📦 espeak: ✅ Disponível
🔹 Versão: eSpeak text-to-speech: 1.48.15  16.Apr.15  Data at: /usr/lib/x86_64-linux-gnu/espeak-data

🧪 Diagnóstico concluído.



In [4]:
# BLOCO 04 - IMPORTS DE BIBLIOTECAS (Atualizado para Multiagentes)

# 📁 Importa todas as bibliotecas necessárias para o funcionamento do Cuidar.AI

# --- Bibliotecas padrão ---
import time                      # Manipulação de tempo
import socket                    # Verificação de conectividade

# --- Google Colab ---
from google.colab import userdata       # Acesso seguro à API key
from google.colab import files          # Para download de arquivos
from IPython.display import Audio, display, clear_output  # Mídia e interface

# --- IA Generativa (Google Generative AI SDKs) ---
import google.generativeai as genai     # SDK v1.5 (modo tradicional, compatibilidade)
from google.genai import types          # SDK v2 - tipos para agentes (Content, Part)
from google import genai as genai_v2    # SDK v2 - interface Client

# --- Google ADK (Agentes Multiagentes) ---
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search

# --- Áudio e Voz ---
from gtts import gTTS                   # Síntese de voz online
from pydub import AudioSegment          # Manipulação de áudio
import pyttsx3                          # Voz offline (fallback)

# --- Utilitários diversos ---
from tqdm import tqdm                   # Barra de progresso
import ipywidgets as widgets            # Interface interativa
import warnings                         # Supressão de alertas

# --- Tratamento de erros ---
from google.api_core.exceptions import GoogleAPIError


In [5]:
 # BLOCO 05 - Verificação de versões dos SDKs

 # ✅ Verificando a versão do SDK google-generativeai (v1.5)
 try:
     print(f"✅ google-generativeai versão: {genai.__version__}")
 except Exception as e:
     print(f"⚠️ Não foi possível verificar a versão do google-generativeai: {e}")
  # ✅ Verificando a versão do SDK google-genai (v2)
 try:
     import google.genai as genai_v2
     print(f"✅ google-genai versão: {genai_v2.__version__}")
 except Exception as e:
     print(f"⚠️ Não foi possível verificar a versão do google-genai (v2): {e}")


✅ google-generativeai versão: 0.8.5
✅ google-genai versão: 1.15.0


In [6]:
# BLOCO 06

import glob
from datetime import datetime
import zipfile
from google.genai import Client, types
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from IPython.display import Markdown
import textwrap

# 🗑️ Função para limpar logs antigos
def clean_old_logs(log_pattern="erros_cuidar_*.txt", max_age_days=30):
    try:
        for old_log in glob.glob(log_pattern):
            try:
                log_date = datetime.strptime(old_log[-19:-4], '%Y%m%d_%H%M%S')
                if (datetime.now() - log_date).days > max_age_days:
                    os.remove(old_log)
                    print(f"🗑️ Log antigo removido: {old_log}")
            except (ValueError, OSError) as e:
                print(f"⚠️ Ignorando log {old_log}: {e}")
    except Exception as e:
        print(f"❌ Erro ao limpar logs: {e}")

# 📋 Função para rotação de log
def rotate_log_file(log_file="erros_cuidar.txt", max_size=1_000_000):
    try:
        if os.path.exists(log_file) and os.path.getsize(log_file) > max_size:
            timestamp = time.strftime('%Y%m%d_%H%M%S')
            new_log = f"{log_file.rsplit('.', 1)[0]}_{timestamp}.txt"
            os.rename(log_file, new_log)
            print(f"📜 Arquivo de log rotacionado para {new_log}")
            zip_name = f"{log_file.rsplit('.', 1)[0]}_{timestamp}.zip"
            with zipfile.ZipFile(zip_name, "w", zipfile.ZIP_DEFLATED) as zf:
                zf.write(new_log)
            os.remove(new_log)
            print(f"📦 Log compactado para {zip_name}")
    except OSError as e:
        print(f"❌ Erro ao rotacionar log: {e}")

# 🗑️ Limpa logs antigos no início
clean_old_logs()

# 📋 Validação inicial da chave API
try:
    api_key = userdata.get('GOOGLE_API_KEY') or ""
    if not api_key or not api_key.startswith('AIza'):
        error_msg = "Chave API inválida ou não encontrada no Colab Secrets."
        rotate_log_file()
        with open("erros_cuidar.txt", "a", encoding="utf-8") as f:
            f.write(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Erro: {error_msg}\n")
        raise ValueError(error_msg)
except Exception as e:
    error_msg = f"Erro ao acessar Colab Secrets: {e}"
    rotate_log_file()
    with open("erros_cuidar.txt", "a", encoding="utf-8") as f:
        f.write(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] {error_msg}\n")
    raise

# 🔄 Configuração da API v1.5 com tentativas
for attempt in range(3):
    try:
        genai.configure(api_key=api_key)
        print("✅ API do Gemini v1.5 configurada com sucesso!")
        break
    except GoogleAPIError as e:
        error_msg = f"Erro na API do Gemini v1.5: {e}"
        print(f"❌ {error_msg}. Tentando novamente... (Tentativa {attempt + 1}/3)")
        rotate_log_file()
        with open("erros_cuidar.txt", "a", encoding="utf-8") as f:
            f.write(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] {error_msg}\n")
        if attempt == 2:
            raise
        time.sleep(2)

# ✅ Configuração do Gemini SDK v2 (Client para uso com agentes)
try:
    client = Client(api_key=api_key)
    print("✅ Cliente Gemini v2 (google-genai) configurado com sucesso!")
except Exception as e:
    error_msg = f"Erro ao configurar o Client do SDK v2: {e}"
    rotate_log_file()
    print(f"❌ {error_msg}")
    with open("erros_cuidar.txt", "a", encoding="utf-8") as f:
        f.write(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] {error_msg}\n")
    raise

# 🎙️ Teste do gTTS
try:
    tts = gTTS(text="Teste", lang="pt")
    print("✅ gTTS inicializado com sucesso!")
except Exception as e:
    error_msg = f"Erro no gTTS: {e}"
    print(f"❌ {error_msg}")
    rotate_log_file()
    with open("erros_cuidar.txt", "a", encoding="utf-8") as f:
        f.write(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] {error_msg}\n")
    raise

# ✅ Função auxiliar para execução de agentes com ADK
def call_agent(agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text + "\n"
    return final_response.strip()

# ✅ Função utilitária para exibir texto formatado no Colab
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

print("✅ Configuração inicial concluída. Pronto para usar o Cuidar.AI com agentes!")


✅ API do Gemini v1.5 configurada com sucesso!
✅ Cliente Gemini v2 (google-genai) configurado com sucesso!
✅ gTTS inicializado com sucesso!
✅ Configuração inicial concluída. Pronto para usar o Cuidar.AI com agentes!


In [7]:
# BLOCO 07 - Verificação de Conectividade com a Internet

# 🌐 Função que testa conexão com a internet tentando se conectar ao Google
def is_online(test_host="www.google.com", port=80, timeout=5) -> bool:
    try:
        socket.create_connection((test_host, port), timeout=timeout)
        return True
    except OSError:
        return False

# 📡 Executa o teste de conectividade
if is_online():
    print("✅ Conectado à internet!")
else:
    print("❌ Sem conexão com a internet. Algumas funcionalidades podem não funcionar.")


✅ Conectado à internet!


In [8]:
# BLOCO 08 - Inicialização do mecanismo de voz (pyttsx3 com fallback)

engine = None
voices = []

try:
    engine = pyttsx3.init()
    print("✅ pyttsx3 inicializado com sucesso!")

    # 🎙️ Obtendo vozes disponíveis
    voices = engine.getProperty('voices')
    if voices:
        print(f"🔍 {len(voices)} vozes disponíveis para uso offline.")
    else:
        print("⚠️ Nenhuma voz disponível no pyttsx3. Usando gTTS como fallback.")
except Exception as e:
    print(f"⚠️ pyttsx3 não funcional ou não suportado no Colab: {e}. Usando gTTS como fallback.")
    voices = []

# 📋 Menu interativo para exibir informações das vozes (somente fora do Colab)
if voices:
    options = [f"{i+1}. {voice.name}" for i, voice in enumerate(voices)]
    dropdown = widgets.Dropdown(options=options, description='Voz:', style={'description_width': 'initial'})

    def exibir_voz(change):
        i = int(change['new'].split('.')[0]) - 1
        voice = voices[i]
        clear_output()
        display(dropdown)
        print(f"🆔 ID: {voice.id}")
        print(f"🎙️ Nome: {voice.name}")
        print(f"🌐 Idioma(s): {voice.languages if hasattr(voice, 'languages') else 'N/A'}")

    dropdown.observe(exibir_voz, names='value')
    display(dropdown)
else:
    print("ℹ️ Menu de vozes não exibido devido à falta de suporte do pyttsx3.")


✅ pyttsx3 inicializado com sucesso!
🔍 75 vozes disponíveis para uso offline.


Dropdown(description='Voz:', options=('1. afrikaans', '2. aragonese', '3. bulgarian', '4. bengali', '5. bosnia…

In [9]:
# BLOCO 09 - Função para gerar e reproduzir áudio com gTTS
def falar_texto(texto, lang='pt', interativo=True):
    try:
        # 🧠 Validação leve de idioma (padrões suportados pelo gTTS)
        idiomas_suportados = ['pt', 'en', 'es', 'fr', 'de', 'it', 'ru', 'ja']
        lang = lang.lower()
        if lang not in idiomas_suportados:
            print(f"⚠️ Idioma '{lang}' não suportado. Usando 'pt' por padrão.")
            lang = 'pt'

        # 🔁 Divide o texto em partes menores para gerar múltiplos arquivos se necessário
        max_chars = 500
        partes = [texto[i:i + max_chars] for i in range(0, len(texto), max_chars)]
        print(f"\n📢 Texto dividido em {len(partes)} partes para áudio.")
        audio_final = AudioSegment.empty()

        for idx, parte in enumerate(tqdm(partes, desc="Gerando áudio", unit="parte")):
            tts = gTTS(text=parte, lang=lang)
            tts.save(f"parte_{idx}.mp3")
            audio = AudioSegment.from_mp3(f"parte_{idx}.mp3")
            audio_final += audio
            os.remove(f"parte_{idx}.mp3")

        # 💾 Exporta o áudio completo
        audio_final.export("fala.mp3", format="mp3")
        print("✅ Áudio gerado: fala.mp3")
        display(Audio("fala.mp3", autoplay=False))

        num_palavras = len(texto.split())
        duracao_estimada = (num_palavras / 150) * 60
        print(f"🔊 Áudio disponível (duração estimada: {duracao_estimada:.1f} segundos).")

        # 💾 Pergunta ao usuário se deseja baixar (modo interativo)
        if interativo:
            try:
                if input("💾 Deseja baixar o áudio? (s/n): ").strip().lower() == "s":
                    files.download("fala.mp3")
            except Exception:
                print("⚠️ Erro ao capturar input para download.")

        # 🧹 Tenta excluir o arquivo após uso
        try:
            time.sleep(1)
            os.remove("fala.mp3")
        except Exception as e:
            print(f"⚠️ Erro ao excluir o áudio: {e}")

        print("\n✅ Áudio concluído.\n---")
    except Exception as e:
        error_msg = f"Erro ao gerar áudio: {e}"
        print(f"\n❌ {error_msg}\n")
        rotate_log_file()
        with open("erros_cuidar.txt", "a", encoding="utf-8") as f:
            f.write(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] {error_msg}\n")


In [10]:
# BLOCO 10 - Função para gerar áudio offline com pyttsx3
def falar_texto_alternativo(texto, lang='pt', engine_override=None, voices_override=None, quiet=False):
    if is_colab:
        if not quiet:
            print("⚠️ pyttsx3 não suportado no Colab. Usando gTTS como fallback.")
        return False
    try:
        engine = engine_override or pyttsx3.init()
        engine.setProperty('rate', 150)
        voices = voices_override or engine.getProperty('voices')

        lang_map = {
            'pt': ['portuguese', 'brazil'],
            'es': ['spanish'],
            'en': ['english'],
            'fr': ['french'],
            'it': ['italian'],
            'de': ['german'],
            'ja': ['japanese'],
            'ru': ['russian']
        }

        voice_found = False
        for voice in voices:
            if any(keyword in voice.name.lower() for keyword in lang_map.get(lang, ['portuguese'])):
                engine.setProperty('voice', voice.id)
                voice_found = True
                break

        if not voice_found and not quiet:
            print(f"\n⚠️ Voz para idioma '{lang}' não encontrada. Usando voz padrão.\n")

        engine.say(texto)
        engine.runAndWait()
        return True
    except Exception as e:
        error_msg = f"Erro ao gerar áudio com pyttsx3: {e}"
        if not quiet:
            print(f"\n❌ {error_msg}\n")
        rotate_log_file()
        with open("erros_cuidar.txt", "a", encoding="utf-8") as f:
            f.write(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] {error_msg}\n")
        return False


In [11]:
# BLOCO 11 - Função de diagnóstico/testes iniciais do sistema
def testar_setup_cuidar_ai(quiet=False):
    print("\n🧪 Iniciando testes para configuração do Cuidar.AI...")

    # Teste 1: Bibliotecas
    if not quiet:
        print("\n📋 Teste 1: Verificação de bibliotecas")
    try:
        for modulo, nome in bibliotecas.items():
            if importlib.util.find_spec(modulo):
                if not quiet:
                    print(f"✅ {nome} detectado.")
            else:
                print(f"❌ {nome} não encontrado.")
        if not quiet:
            print("✅ Teste 1 concluído: Bibliotecas OK.")
    except Exception as e:
        print(f"❌ Teste 1 falhou: {e}")
        logar_erro(f"Teste 1 falhou: {e}")

    # Teste 2: API
    if not quiet:
        print("\n🔄 Teste 2: Configuração da API")
    try:
        api_key = userdata.get('GOOGLE_API_KEY') or ""
        if not api_key.startswith('AIza'):
            raise ValueError("Chave API inválida ou ausente.")
        genai.configure(api_key=api_key)
        if not quiet:
            print("✅ Teste 2 concluído: API configurada.")
    except Exception as e:
        print(f"❌ Teste 2 falhou: {e}")
        logar_erro(f"Teste 2 falhou: {e}")

    # Teste 3: Conexão
    if not quiet:
        print("\n🌐 Teste 3: Verificação de conexão")
    try:
        if is_online():
            if not quiet:
                print("✅ Conexão detectada.")
        else:
            print("⚠️ Sem conexão, mas teste prossegue.")
        if not quiet:
            print("✅ Teste 3 concluído: Conexão verificada.")
    except Exception as e:
        print(f"❌ Teste 3 falhou: {e}")
        logar_erro(f"Teste 3 falhou: {e}")

    # Teste 4: gTTS
    if not quiet:
        print("\n🎙️ Teste 4: gTTS")
    try:
        falar_texto("Teste do Cuidar.AI", lang='pt', interativo=False)
        if not quiet:
            print("✅ Teste 4 concluído: gTTS funcional.")
    except Exception as e:
        print(f"❌ Teste 4 falhou: {e}")
        logar_erro(f"Teste 4 falhou: {e}")

    # Teste 5: pyttsx3
    if not quiet:
        print("\n🎙️ Teste 5: pyttsx3")
    try:
        if is_colab:
            if not quiet:
                print("ℹ️ Teste pulado: pyttsx3 não suportado no Colab.")
        else:
            if falar_texto_alternativo("Teste offline", lang='pt', engine_override=engine, voices_override=voices, quiet=quiet):
                if not quiet:
                    print("✅ Áudio offline gerado.")
            else:
                print("⚠️ Falha no pyttsx3, mas prossegue.")
        if not quiet:
            print("✅ Teste 5 concluído.")
    except Exception as e:
        print(f"❌ Teste 5 falhou: {e}")
        logar_erro(f"Teste 5 falhou: {e}")

    # Teste 6: Menu de vozes
    if not quiet:
        print("\n📋 Teste 6: Verificação de vozes disponíveis")
    try:
        if voices:
            if not quiet:
                print("✅ Vozes offline disponíveis.")
        else:
            print("⚠️ Nenhuma voz encontrada.")
        if not quiet:
            print("✅ Teste 6 concluído.")
    except Exception as e:
        print(f"❌ Teste 6 falhou: {e}")
        logar_erro(f"Teste 6 falhou: {e}")

    if not quiet:
        print("\n✅ Testes finalizados.\n")

# Executar (manual)
# testar_setup_cuidar_ai()


# EATAPA 02 - Módulos funcionais

In [12]:
# BLOCO 12 - Função auxiliar para registro de erros
def logar_erro(mensagem, log_file="erros_cuidar.txt", quiet=False):
    rotate_log_file(log_file=log_file)
    try:
        with open(log_file, "a", encoding="utf-8") as f:
            f.write(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] {mensagem}\n")
        if not quiet:
            print(f"❌ {mensagem}")
    except Exception as e:
        if not quiet:
            print(f"❌ Erro ao registrar log: {e}")


In [13]:
# BLOCO 13 - Função responder_com_voz adaptada para multiagente
def responder_com_voz(pergunta, chat, lang='pt', use_gui=False, output_widget=None, agente_nome=None, salvar=True, ouvir=True):
    try:
        resposta = chat.send_message(pergunta)
        resposta_texto = resposta.text

        # Rótulo por agente (opcional, para logs)
        rotulo_agente = f"[{agente_nome}]" if agente_nome else ""

        if use_gui and output_widget:
            with output_widget:
                clear_output()
                print("\n📜 Texto gerado pelo Gemini:")
                print("══════════════════════")
                print(resposta_texto)
                print("══════════════════════\n")

                audio_button = widgets.Button(description="🔊 Ouvir Resposta", button_style="primary")
                save_button = widgets.Button(description="📄 Salvar Resposta", button_style="success")
                local_output = widgets.Output()

                def on_audio_clicked(b):
                    with local_output:
                        clear_output()
                        try:
                            use_offline = not is_online() or lang == "pt"
                            if use_offline:
                                if not falar_texto_alternativo(resposta_texto, lang=lang):
                                    print("\n⚠️ Falha no pyttsx3. Usando gTTS.\n")
                                    falar_texto(resposta_texto, lang=lang)
                                else:
                                    print(f"\n✅ Áudio em {lang} reproduzido com sucesso.\n")
                            else:
                                falar_texto(resposta_texto, lang=lang)
                                print(f"\n✅ Áudio em {lang} reproduzido com sucesso.\n")
                        except Exception as e:
                            logar_erro(f"{rotulo_agente} Erro ao gerar áudio: {e}")

                def on_save_clicked(b):
                    with local_output:
                        clear_output()
                        try:
                            with open("respostas_cuidar.txt", "a", encoding="utf-8") as f:
                                f.write(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] {rotulo_agente} Pergunta: {pergunta}\nResposta: {resposta_texto}\n\n")
                            print("\n✅ Resposta salva em respostas_cuidar.txt\n")
                            files.download("respostas_cuidar.txt")
                        except Exception as e:
                            logar_erro(f"{rotulo_agente} Erro ao salvar resposta: {e}")

                audio_button.on_click(on_audio_clicked)
                save_button.on_click(on_save_clicked)
                display(widgets.VBox([audio_button, save_button, local_output]))
        else:
            print("\n📜 Texto gerado pelo Gemini:")
            print("══════════════════════")
            print(resposta_texto)
            print("══════════════════════\n")

            if ouvir:
                try:
                    escolha_audio = input("🔊 Deseja ouvir a resposta? (s/n): ").lower()
                    if escolha_audio == "s":
                        use_offline = not is_online() or lang == "pt" or input("📴 Usar voz offline (pyttsx3)? (s/n): ").lower() == "s"
                        if use_offline:
                            if not falar_texto_alternativo(resposta_texto, lang=lang):
                                print("\n⚠️ Falha no pyttsx3. Usando gTTS como fallback.\n")
                                falar_texto(resposta_texto, lang=lang)
                            else:
                                print(f"\n✅ Áudio em {lang} reproduzido com sucesso.\n")
                        else:
                            falar_texto(resposta_texto, lang=lang)
                            print(f"\n✅ Áudio em {lang} reproduzido com sucesso.\n")
                except Exception as e:
                    logar_erro(f"{rotulo_agente} Erro ao gerar áudio: {e}")

            if salvar:
                try:
                    escolha_salvar = input("📄 Deseja salvar a resposta em um arquivo? (s/n): ").lower()
                    if escolha_salvar == "s":
                        with open("respostas_cuidar.txt", "a", encoding="utf-8") as f:
                            f.write(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] {rotulo_agente} Pergunta: {pergunta}\nResposta: {resposta_texto}\n\n")
                        print("\n✅ Resposta salva em respostas_cuidar.txt\n")
                        files.download("respostas_cuidar.txt")
                except Exception as e:
                    logar_erro(f"{rotulo_agente} Erro ao salvar resposta: {e}")

        return resposta_texto

    except GoogleAPIError as e:
        logar_erro(f"{rotulo_agente} Erro na API do Gemini: {e}")
        if use_gui and output_widget:
            with output_widget:
                print(f"\n❌ {e}\nTente novamente ou verifique sua conexão.")
        else:
            print(f"\n❌ {e}\nTente novamente ou verifique sua conexão.")
        return None
    except Exception as e:
        logar_erro(f"{rotulo_agente} Erro inesperado: {e}")
        if use_gui and output_widget:
            with output_widget:
                print(f"\n❌ {e}\n")
        else:
            print(f"\n❌ {e}\n")
        return None


In [14]:
# BLOCO 14 - Função genérica para consultas com histórico (multiagente)
def consultar_generico(
    pergunta,
    prompt_template,
    chat,
    historico,
    lang='pt',
    use_gui=False,
    output_widget=None,
    historico_labels=("Pergunta", "Resposta"),
    agente_nome=None,
    salvar=True,
    ouvir=True
):
    """
    Executa uma consulta genérica usando um template de prompt e registra no histórico.

    Parâmetros:
    - pergunta: string com a pergunta do usuário
    - prompt_template: função que gera o prompt com base na pergunta
    - chat: objeto de sessão Gemini
    - historico: lista onde o par (pergunta, resposta) será salvo
    - lang: idioma para síntese de voz
    - use_gui: modo com interface gráfica
    - output_widget: área de saída da interface
    - historico_labels: rótulos opcionais para armazenamento
    - agente_nome: nome do módulo/agente para logs
    - salvar: se True, salva a resposta gerada
    - ouvir: se True, ativa pergunta de áudio
    """
    try:
        prompt = prompt_template(pergunta)
        resposta = responder_com_voz(
            pergunta=prompt,
            chat=chat,
            lang=lang,
            use_gui=use_gui,
            output_widget=output_widget,
            agente_nome=agente_nome,
            salvar=salvar,
            ouvir=ouvir
        )
        if resposta and historico is not None:
            historico.append((historico_labels[0] + ": " + pergunta, historico_labels[1] + ": " + resposta))
        return resposta
    except Exception as e:
        logar_erro(f"[{agente_nome or 'Genérico'}] Erro ao processar consulta: {e}")
        return None


In [15]:
# BLOCO 15

# 🧩 Função utilitária para montar interfaces com widgets (multiagente-friendly)
def montar_interface(pergunta_label, on_submit, on_exit, on_history=None, placeholder="Digite aqui", texto_enviar="Enviar"):
    input_text = widgets.Text(description=pergunta_label, placeholder=placeholder, style={'description_width': 'initial'})
    submit_button = widgets.Button(description=texto_enviar, button_style="primary")
    exit_button = widgets.Button(description="Sair", button_style="danger")

    elements = [input_text, submit_button]

    if on_history:
        history_button = widgets.Button(description="Histórico", button_style="info")
        elements.append(history_button)
        history_button.on_click(on_history)

    elements.append(exit_button)

    submit_button.on_click(lambda b: on_submit(input_text))
    exit_button.on_click(on_exit)

    return widgets.VBox(elements), input_text



In [16]:
# BLOCO 16

# 🩺 Módulo Saúde Preventiva com interface unificada
# 🤖 Agente: Saúde Preventiva
# Este módulo atua como um agente autônomo no sistema multiagente do Cuidar.AI,
# focado em responder perguntas sobre saúde preventiva com suporte a CLI e GUI.

def saude_preventiva_gemini(chat, use_gui=False, output_widget=None):
    historico = []

    def prompt_template(pergunta):
        return f"Responda à seguinte pergunta sobre saúde preventiva: '{pergunta}'"

    if use_gui:
        local_output = widgets.Output()

        def on_submit(input_widget):
            with local_output:
                clear_output()
                pergunta = input_widget.value.strip()
                if not pergunta:
                    print("⚠️ Por favor, digite uma pergunta.")
                    return
                consultar_generico(pergunta, prompt_template, chat, historico, use_gui=True, output_widget=local_output)
                input_widget.value = ""

        def on_history(b):
            with local_output:
                clear_output()
                if not historico:
                    print("📜 Nenhum histórico disponível.")
                else:
                    print("\n📜 Histórico de Interações:")
                    print("══════════════════════")
                    for idx, (p, r) in enumerate(historico, 1):
                        print(f"{idx}. Pergunta: {p}\n   Resposta: {r}\n")
                    print("══════════════════════\n")

        def on_exit(b):
            with local_output:
                clear_output()
                print("🏁 Módulo Saúde Preventiva encerrado.")
            if output_widget:
                with output_widget:
                    clear_output()
                    print("🏁 Módulo Saúde Preventiva encerrado.")

        ui_box, input_text = montar_interface("Pergunta:", on_submit, on_exit, on_history)
        display(widgets.VBox([ui_box, local_output]))

    else:
        print("\n🔎 Módulo Saúde Preventiva - Digite 'fim' para encerrar ou 'histórico' para ver interações.\n")
        while True:
            try:
                pergunta = input("❓ Você: ").strip()
                if pergunta.lower() == "fim":
                    print("\n🏁 Módulo encerrado.\n")
                    break
                elif pergunta.lower() == "histórico":
                    if not historico:
                        print("📜 Nenhum histórico disponível.")
                    else:
                        print("\n📜 Histórico de Interações:")
                        print("══════════════════════")
                        for idx, (p, r) in enumerate(historico, 1):
                            print(f"{idx}. Pergunta: {p}\n   Resposta: {r}\n")
                        print("══════════════════════\n")
                    continue
                if not pergunta:
                    print("⚠️ Por favor, digite uma pergunta.")
                    continue
                consultar_generico(pergunta, prompt_template, chat, historico)
            except (EOFError, KeyboardInterrupt):
                logar_erro("Entrada interrompida pelo usuário.")
                print("\n🏁 Módulo encerrado.\n")
                break


In [17]:
# BLOCO 17

# 🤖 Agente: Serviços Públicos
# Explica passo a passo como acessar serviços governamentais (CPF, RG, benefícios etc.)

def servicos_publicos_gemini(chat, use_gui=False, output_widget=None):
    historico = []

    def prompt_template(pergunta):
        return (
            f"Explique de forma clara como o cidadão pode acessar o seguinte serviço público: '{pergunta}'. "
            "Dê detalhes úteis, etapas e onde buscar mais informações confiáveis."
        )

    if use_gui:
        local_output = widgets.Output()

        def on_submit(input_widget):
            with local_output:
                clear_output()
                pergunta = input_widget.value.strip()
                if not pergunta:
                    print("⚠️ Por favor, digite um serviço público.")
                    return
                consultar_generico(
                    pergunta,
                    prompt_template,
                    chat,
                    historico,
                    use_gui=True,
                    output_widget=local_output,
                    agente_nome="Serviços Públicos",
                    salvar=True,
                    ouvir=False
                )
                input_widget.value = ""

        def on_history(_):
            with local_output:
                clear_output()
                if not historico:
                    print("📜 Nenhum histórico disponível.")
                else:
                    print("\n📜 Histórico de Interações:")
                    print("══════════════════════")
                    for idx, (p, r) in enumerate(historico, 1):
                        print(f"{idx}. Serviço: {p}\n   Resposta: {r}\n")
                    print("══════════════════════\n")

        def on_exit(_):
            for child in ui_box.children:
                if hasattr(child, 'disabled'):
                    child.disabled = True
            with local_output:
                clear_output()
                print("🏁 Módulo Serviços Públicos encerrado.")
            if output_widget:
                with output_widget:
                    clear_output()
                    print("🏁 Módulo Serviços Públicos encerrado.")

        ui_box, input_text = montar_interface(
            pergunta_label="Serviço:",
            on_submit=on_submit,
            on_exit=on_exit,
            on_history=on_history,
            placeholder="Ex: Como tirar CPF",
            texto_enviar="Consultar"
        )
        display(widgets.VBox([ui_box, local_output]))

    else:
        print("\n🏛️ Módulo de Serviços Públicos - Digite 'fim' para encerrar ou 'histórico' para ver interações.\n")
        while True:
            try:
                pergunta = input("❓ Qual serviço público deseja saber? ").strip()
                if pergunta.lower() == "fim":
                    print("\n🏁 Módulo encerrado.\n")
                    break
                elif pergunta.lower() == "histórico":
                    if not historico:
                        print("📜 Nenhum histórico disponível.")
                    else:
                        print("\n📜 Histórico de Interações:")
                        print("══════════════════════")
                        for idx, (p, r) in enumerate(historico, 1):
                            print(f"{idx}. Serviço: {p}\n   Resposta: {r}\n")
                        print("══════════════════════\n")
                    continue
                if not pergunta:
                    print("⚠️ Por favor, digite um serviço público.")
                    continue

                consultar_generico(
                    pergunta,
                    prompt_template,
                    chat,
                    historico,
                    agente_nome="Serviços Públicos",
                    salvar=False,
                    ouvir=False
                )

            except (EOFError, KeyboardInterrupt):
                logar_erro("Serviços Públicos: entrada interrompida pelo usuário.")
                print("\n🏁 Módulo encerrado.\n")
                break


In [18]:
# BLOCO 18

# Módulo Tradução Cidadã
def traducao_cidada_gemini(chat, use_gui=False, output_widget=None):
    historico = []

    def prompt_template(texto):
        return f"Simplifique o seguinte texto jurídico ou técnico para linguagem acessível ao cidadão comum: '{texto}'"

    if use_gui:
        local_output = widgets.Output()

        def on_submit(input_widget):
            with local_output:
                clear_output()
                texto = input_widget.value.strip()
                if not texto:
                    print("⚠️ Por favor, digite um texto.")
                    return
                consultar_generico(
                    texto,
                    prompt_template,
                    chat,
                    historico,
                    use_gui=True,
                    output_widget=local_output,
                    historico_labels=("Texto", "Simplificação")
                )
                input_widget.value = ""

        def on_history(b):
            with local_output:
                clear_output()
                if not historico:
                    print("📜 Nenhum histórico disponível.")
                else:
                    print("\n📜 Histórico de Interações:")
                    print("══════════════════════")
                    for idx, (p, r) in enumerate(historico, 1):
                        print(f"{idx}. Texto: {p}\n   Simplificação: {r}\n")
                    print("══════════════════════\n")

        def on_exit(b):
            with local_output:
                clear_output()
                print("🏁 Módulo Tradução Cidadã encerrado.")
            if output_widget:
                with output_widget:
                    clear_output()
                    print("🏁 Módulo Tradução Cidadã encerrado.")

        interface, input_text = montar_interface("Texto:", on_submit, on_exit, on_history)
        display(widgets.VBox([interface, local_output]))

    else:
        print("\n🗣️ Módulo Tradução Cidadã - Digite 'fim' para encerrar ou 'histórico' para ver interações.\n")
        while True:
            try:
                texto = input("❓ Qual texto deseja simplificar? ").strip()
                if texto.lower() == "fim":
                    print("\n🏁 Módulo encerrado.\n")
                    break
                elif texto.lower() == "histórico":
                    if not historico:
                        print("📜 Nenhum histórico disponível.")
                    else:
                        print("\n📜 Histórico de Interações:")
                        print("══════════════════════")
                        for idx, (p, r) in enumerate(historico, 1):
                            print(f"{idx}. Texto: {p}\n   Simplificação: {r}\n")
                        print("══════════════════════\n")
                    continue
                if not texto:
                    print("⚠️ Por favor, digite um texto.")
                    continue
                consultar_generico(
                    texto,
                    prompt_template,
                    chat,
                    historico,
                    historico_labels=("Texto", "Simplificação")
                )
            except (EOFError, KeyboardInterrupt):
                logar_erro("Entrada interrompida pelo usuário.")
                print("\n🏁 Módulo encerrado.\n")
                break


In [19]:
# BLOCO 19

# Módulo Tradução para Outras Línguas
def traducao_linguas_gemini(chat, use_gui=False, output_widget=None):
    historico = []
    idioma_map = {
        "espanhol": "es",
        "inglês": "en",
        "francês": "fr",
        "português": "pt",
        "italiano": "it",
        "alemão": "de",
        "japonês": "ja",
        "russo": "ru",
    }

    def prompt_template(frase, idioma):
        return f"Traduza a seguinte frase para o idioma '{idioma}': '{frase}'"

    if use_gui:
        frase_input = widgets.Text(description="Frase:", placeholder="Ex.: Olá, mundo!", style={'description_width': 'initial'})
        idioma_dropdown = widgets.Dropdown(
            options=list(idioma_map.keys()),
            description="Idioma:",
            style={'description_width': 'initial'}
        )
        local_output = widgets.Output()

        def on_submit(input_widget):
            with local_output:
                clear_output()
                frase = frase_input.value.strip()
                idioma = idioma_dropdown.value
                if not frase:
                    print("⚠️ Por favor, insira uma frase válida.")
                    return
                if not idioma:
                    print("⚠️ Por favor, selecione um idioma.")
                    return
                prompt = prompt_template(frase, idioma)
                resposta = responder_com_voz(
                    pergunta=prompt,
                    chat=chat,
                    lang=idioma_map.get(idioma.lower(), "pt"),
                    use_gui=True,
                    output_widget=local_output
                )
                if resposta:
                    historico.append((frase, idioma, resposta))
                    frase_input.value = ""

        def on_exit(b):
            frase_input.disabled = True
            idioma_dropdown.disabled = True
            print("🏁 Módulo Tradução para Outras Línguas encerrado.")
            if output_widget:
                with output_widget:
                    clear_output()
                    print("🏁 Módulo Tradução para Outras Línguas encerrado.")

        def on_history(b):
            with local_output:
                clear_output()
                if not historico:
                    print("📜 Nenhum histórico disponível.")
                else:
                    print("\n📜 Histórico de Interações:")
                    print("══════════════════════")
                    for idx, (p, i, r) in enumerate(historico, 1):
                        print(f"{idx}. Frase: {p}\n   Idioma: {i}\n   Tradução: {r}\n")
                    print("══════════════════════\n")

        interface, _ = montar_interface("Frase:", on_submit, on_exit, on_history)
        interface.children = list(interface.children[:1]) + [idioma_dropdown] + list(interface.children[1:]) + [local_output]
        display(interface)

    else:
        print("\n🌐 Módulo de Tradução para Outras Línguas - Digite 'fim' para encerrar ou 'histórico' para ver interações.")
        print("\nIdiomas com suporte para áudio:")
        for idx, idioma in enumerate(idioma_map.keys(), 1):
            print(f"{idx}. {idioma}")
        while True:
            try:
                frase = input("❓ Frase a traduzir: ").strip()
                if frase.lower() == "fim":
                    print("\n🏁 Módulo encerrado.\n")
                    break
                elif frase.lower() == "histórico":
                    if not historico:
                        print("📜 Nenhum histórico disponível.")
                    else:
                        print("\n📜 Histórico de Interações:")
                        print("══════════════════════")
                        for idx, (p, i, r) in enumerate(historico, 1):
                            print(f"{idx}. Frase: {p}\n   Idioma: {i}\n   Tradução: {r}\n")
                        print("══════════════════════\n")
                    continue
                if not frase:
                    print("\n⚠️ Por favor, insira uma frase válida.\n")
                    continue
                idioma = input("🌎 Para qual idioma deseja traduzir? ").strip()
                if not idioma or not idioma.lower() in idioma_map:
                    print(f"\n⚠️ Idioma '{idioma}' não suportado. Escolha entre: {', '.join(idioma_map.keys())}.\n")
                    continue
                prompt = prompt_template(frase, idioma)
                resposta = responder_com_voz(
                    pergunta=prompt,
                    chat=chat,
                    lang=idioma_map.get(idioma.lower(), "pt")
                )
                if resposta:
                    historico.append((frase, idioma, resposta))
            except (EOFError, KeyboardInterrupt):
                logar_erro("Entrada interrompida pelo usuário.")
                print("\n🏁 Módulo encerrado.\n")
                break


In [20]:
# BLOCO 20
# ✅ Adaptado com montar_interface() + output_widget (GUI)

# Módulo Tradução Cultural Brasileira
def traducao_cultural_gemini(chat, use_gui=False, output_widget=None):
    historico = []

    def prompt_template(texto):
        return f"Explique o seguinte texto usando linguagem padrão. Pode conter expressões regionais ou indígenas: '{texto}'"

    def mostrar_historico():
        if not historico:
            print("📜 Nenhum histórico disponível.")
        else:
            print("\n📜 Histórico de Interações:")
            print("══════════════════════")
            for idx, (p, r) in enumerate(historico, 1):
                print(f"{idx}. Expressão: {p}\n   Explicação: {r}\n")
            print("══════════════════════\n")

    if use_gui:
        local_output = widgets.Output()

        def on_submit(input_widget):
            with local_output:
                clear_output()
                texto = input_widget.value.strip()
                if not texto:
                    print("⚠️ Por favor, digite uma expressão.")
                    return
                consultar_generico(
                    texto,
                    prompt_template,
                    chat,
                    historico,
                    use_gui=True,
                    output_widget=local_output,
                    historico_labels=("Expressão", "Explicação")
                )
                input_widget.value = ""

        def on_history(b):
            with local_output:
                clear_output()
                mostrar_historico()

        def on_exit(b):
            with local_output:
                clear_output()
                print("🏁 Módulo Tradução Cultural Brasileira encerrado.")
            if output_widget:
                with output_widget:
                    clear_output()
                    print("🏁 Módulo Tradução Cultural Brasileira encerrado.")

        interface, input_widget = montar_interface("Expressão:", on_submit, on_exit, on_history)
        display(widgets.VBox([interface, local_output]))

    else:
        print("\n🗺️ Módulo de Tradução Cultural Brasileira - Digite 'fim' para encerrar ou 'histórico' para ver interações.\n")
        while True:
            try:
                texto = input("❓ Digite a frase com expressão regional: ").strip()
                if texto.lower() == "fim":
                    print("\n🏁 Módulo encerrado.\n")
                    break
                elif texto.lower() == "histórico":
                    mostrar_historico()
                    continue
                if not texto:
                    print("⚠️ Por favor, digite uma expressão.")
                    continue
                consultar_generico(
                    texto,
                    prompt_template,
                    chat,
                    historico,
                    historico_labels=("Expressão", "Explicação")
                )
            except (EOFError, KeyboardInterrupt):
                logar_erro("Entrada interrompida pelo usuário.")
                print("\n🏁 Módulo encerrado.\n")
                break


In [21]:
# BLOCO 21

# Módulo Emergência Cidadã
def emergencia_cidada_gemini(chat, use_gui=False, output_widget=None):
    historico = []

    def prompt_template(descricao):
        return (
            f"Situação de emergência relatada: '{descricao}'. "
            "Forneça instruções claras, práticas e oriente sobre serviços como 190, 192, 193, 199."
        )

    def mostrar_historico():
        if not historico:
            print("📜 Nenhum histórico disponível.")
        else:
            print("\n📜 Histórico de Interações:")
            print("══════════════════════")
            for idx, (p, r) in enumerate(historico, 1):
                print(f"{idx}. Emergência: {p}\n   Orientação: {r}\n")
            print("══════════════════════\n")

    if use_gui:
        local_output = widgets.Output()

        def on_submit(input_widget):
            with local_output:
                clear_output()
                descricao = input_widget.value.strip()
                if not descricao:
                    print("⚠️ Por favor, descreva a emergência.")
                    return
                consultar_generico(
                    descricao,
                    prompt_template,
                    chat,
                    historico,
                    use_gui=True,
                    output_widget=local_output,
                    historico_labels=("Emergência", "Orientação")
                )
                input_widget.value = ""

        def on_history(b):
            with local_output:
                clear_output()
                mostrar_historico()

        def on_exit(b):
            with local_output:
                clear_output()
                print("🏁 Módulo Emergência Cidadã encerrado.")
            if output_widget:
                with output_widget:
                    clear_output()
                    print("🏁 Módulo Emergência Cidadã encerrado.")

        interface, input_widget = montar_interface("Emergência:", on_submit, on_exit, on_history)
        display(widgets.VBox([interface, local_output]))

    else:
        print("\n🚨 Módulo de Emergência Cidadã - Digite 'fim' para encerrar ou 'histórico' para ver interações.\n")
        while True:
            try:
                descricao = input("❓ Descreva a emergência: ").strip()
                if descricao.lower() == "fim":
                    print("\n🏁 Módulo encerrado.\n")
                    break
                elif descricao.lower() == "histórico":
                    mostrar_historico()
                    continue
                if not descricao:
                    print("⚠️ Por favor, descreva a emergência.")
                    continue
                consultar_generico(
                    descricao,
                    prompt_template,
                    chat,
                    historico,
                    historico_labels=("Emergência", "Orientação")
                )
            except (EOFError, KeyboardInterrupt):
                logar_erro("Entrada interrompida pelo usuário.")
                print("\n🏁 Módulo encerrado.\n")
                break


In [22]:
# BLOCO 22

# Módulo Segurança Doméstica
def emergencia_domestica_gemini(chat, use_gui=False, output_widget=None):
    historico = []

    def prompt_template(descricao):
        return (
            f"Situação doméstica: '{descricao}'. "
            "Oriente com calma e informe se deve chamar SAMU (192) ou Bombeiros (193)."
        )

    def mostrar_historico():
        if not historico:
            print("📜 Nenhum histórico disponível.")
        else:
            print("\n📜 Histórico de Interações:")
            print("══════════════════════")
            for idx, (p, r) in enumerate(historico, 1):
                print(f"{idx}. Situação: {p}\n   Orientação: {r}\n")
            print("══════════════════════\n")

    if use_gui:
        local_output = widgets.Output()

        def on_submit(input_widget):
            with local_output:
                clear_output()
                descricao = input_widget.value.strip()
                if not descricao:
                    print("⚠️ Por favor, descreva a situação.")
                    return
                consultar_generico(
                    descricao,
                    prompt_template,
                    chat,
                    historico,
                    use_gui=True,
                    output_widget=local_output,
                    historico_labels=("Situação", "Orientação")
                )
                input_widget.value = ""

        def on_history(b):
            with local_output:
                clear_output()
                mostrar_historico()

        def on_exit(b):
            with local_output:
                clear_output()
                print("🏁 Módulo Segurança Doméstica encerrado.")
            if output_widget:
                with output_widget:
                    clear_output()
                    print("🏁 Módulo Segurança Doméstica encerrado.")

        interface, input_widget = montar_interface("Situação:", on_submit, on_exit, on_history)
        display(widgets.VBox([interface, local_output]))

    else:
        print("\n🏠 Módulo Segurança Doméstica - Digite 'fim' para encerrar ou 'histórico' para ver interações.\n")
        while True:
            try:
                descricao = input("❓ Situação doméstica: ").strip()
                if descricao.lower() == "fim":
                    print("\n🏁 Módulo encerrado.\n")
                    break
                elif descricao.lower() == "histórico":
                    mostrar_historico()
                    continue
                if not descricao:
                    print("⚠️ Por favor, descreva a situação.")
                    continue
                consultar_generico(
                    descricao,
                    prompt_template,
                    chat,
                    historico,
                    historico_labels=("Situação", "Orientação")
                )
            except (EOFError, KeyboardInterrupt):
                logar_erro("Entrada interrompida pelo usuário.")
                print("\n🏁 Módulo encerrado.\n")
                break


In [23]:
# BLOCO 23

# Módulo Emergência Infantil
def emergencia_infantil_gemini(chat, use_gui=False, output_widget=None):
    historico = []

    def prompt_template(descricao):
        return (
            f"Emergência com criança/bebê: '{descricao}'. "
            "Oriente com clareza e cuidado. Indique se deve acionar o SAMU (192)."
        )

    def mostrar_historico():
        if not historico:
            print("📜 Nenhum histórico disponível.")
        else:
            print("\n📜 Histórico de Interações:")
            print("══════════════════════")
            for idx, (p, r) in enumerate(historico, 1):
                print(f"{idx}. Situação: {p}\n   Orientação: {r}\n")
            print("══════════════════════\n")

    if use_gui:
        local_output = widgets.Output()

        def on_submit(input_widget):
            with local_output:
                clear_output()
                descricao = input_widget.value.strip()
                if not descricao:
                    print("⚠️ Por favor, descreva a situação.")
                    return
                consultar_generico(
                    descricao,
                    prompt_template,
                    chat,
                    historico,
                    use_gui=True,
                    output_widget=local_output,
                    historico_labels=("Situação", "Orientação")
                )
                input_widget.value = ""

        def on_history(b):
            with local_output:
                clear_output()
                mostrar_historico()

        def on_exit(b):
            with local_output:
                clear_output()
                print("🏁 Módulo Emergência Infantil encerrado.")
            if output_widget:
                with output_widget:
                    clear_output()
                    print("🏁 Módulo Emergência Infantil encerrado.")

        interface, input_widget = montar_interface("Situação:", on_submit, on_exit, on_history)
        display(widgets.VBox([interface, local_output]))

    else:
        print("\n👶 Módulo Emergência Infantil - Digite 'fim' para encerrar ou 'histórico' para ver interações.\n")
        while True:
            try:
                descricao = input("❓ Situação com criança ou bebê: ").strip()
                if descricao.lower() == "fim":
                    print("\n🏁 Módulo encerrado.\n")
                    break
                elif descricao.lower() == "histórico":
                    mostrar_historico()
                    continue
                if not descricao:
                    print("⚠️ Por favor, descreva a situação.")
                    continue
                consultar_generico(
                    descricao,
                    prompt_template,
                    chat,
                    historico,
                    historico_labels=("Situação", "Orientação")
                )
            except (EOFError, KeyboardInterrupt):
                logar_erro("Entrada interrompida pelo usuário.")
                print("\n🏁 Módulo encerrado.\n")
                break


In [24]:
# BLOCO 24

# Módulo Apoio Emocional
def apoio_psicologico_gemini(chat, use_gui=False, output_widget=None):
    historico = []

    def prompt_template(mensagem):
        return (
            f"Mensagem recebida: '{mensagem}'. "
            "Responda com acolhimento, empatia e oriente sobre o CVV (188) se necessário."
        )

    def mostrar_historico():
        if not historico:
            print("📜 Nenhum histórico disponível.")
        else:
            print("\n📜 Histórico de Interações:")
            print("══════════════════════")
            for idx, (p, r) in enumerate(historico, 1):
                print(f"{idx}. Mensagem: {p}\n   Resposta: {r}\n")
            print("══════════════════════\n")

    if use_gui:
        local_output = widgets.Output()

        def on_submit(input_widget):
            with local_output:
                clear_output()
                mensagem = input_widget.value.strip()
                if not mensagem:
                    print("⚠️ Por favor, digite uma mensagem.")
                    return
                consultar_generico(
                    mensagem,
                    prompt_template,
                    chat,
                    historico,
                    use_gui=True,
                    output_widget=local_output,
                    historico_labels=("Mensagem", "Resposta")
                )
                input_widget.value = ""

        def on_history(b):
            with local_output:
                clear_output()
                mostrar_historico()

        def on_exit(b):
            with local_output:
                clear_output()
                print("🏁 Módulo Apoio Emocional encerrado.")
            if output_widget:
                with output_widget:
                    clear_output()
                    print("🏁 Módulo Apoio Emocional encerrado.")

        interface, input_widget = montar_interface("Mensagem:", on_submit, on_exit, on_history)
        display(widgets.VBox([interface, local_output]))

    else:
        print("\n💙 Módulo Apoio Emocional - Digite 'fim' para encerrar ou 'histórico' para ver interações.\n")
        while True:
            try:
                mensagem = input("❓ Como está se sentindo? ").strip()
                if mensagem.lower() == "fim":
                    print("\n🏁 Módulo encerrado.\n")
                    break
                elif mensagem.lower() == "histórico":
                    mostrar_historico()
                    continue
                if not mensagem:
                    print("⚠️ Por favor, digite uma mensagem.")
                    continue
                consultar_generico(
                    mensagem,
                    prompt_template,
                    chat,
                    historico,
                    historico_labels=("Mensagem", "Resposta")
                )
            except (EOFError, KeyboardInterrupt):
                logar_erro("Entrada interrompida pelo usuário.")
                print("\n🏁 Módulo encerrado.\n")
                break


In [25]:
# BLOCO 25

# Módulo Passa Tempo
def passatempo_gemini(chat, use_gui=False, output_widget=None):
    historico = []

    def prompt_template(pedido):
        return f"Crie um passatempo leve, divertido e educativo. Pedido: '{pedido}'"

    def mostrar_historico():
        if not historico:
            print("📜 Nenhum histórico disponível.")
        else:
            print("\n📜 Histórico de Interações:")
            print("══════════════════════")
            for idx, (p, r) in enumerate(historico, 1):
                print(f"{idx}. Pedido: {p}\n   Resposta: {r}\n")
            print("══════════════════════\n")

    if use_gui:
        local_output = widgets.Output()

        def on_submit(input_widget):
            with local_output:
                clear_output()
                pedido = input_widget.value.strip()
                if not pedido:
                    print("⚠️ Por favor, digite um passatempo.")
                    return
                consultar_generico(
                    pedido,
                    prompt_template,
                    chat,
                    historico,
                    use_gui=True,
                    output_widget=local_output,
                    historico_labels=("Pedido", "Resposta")
                )
                input_widget.value = ""

        def on_history(b):
            with local_output:
                clear_output()
                mostrar_historico()

        def on_exit(b):
            with local_output:
                clear_output()
                print("🏁 Módulo Passa Tempo encerrado.")
            if output_widget:
                with output_widget:
                    clear_output()
                    print("🏁 Módulo Passa Tempo encerrado.")

        interface, input_widget = montar_interface("Passatempo:", on_submit, on_exit, on_history)
        display(widgets.VBox([interface, local_output]))

    else:
        print("\n🎲 Módulo Passa Tempo - Digite 'fim' para encerrar ou 'histórico' para ver interações.\n")
        while True:
            try:
                pedido = input("❓ Que tipo de passatempo você deseja? (piada, curiosidade, desafio, etc.): ").strip()
                if pedido.lower() == "fim":
                    print("\n🏁 Módulo encerrado.\n")
                    break
                elif pedido.lower() == "histórico":
                    mostrar_historico()
                    continue
                if not pedido:
                    print("⚠️ Por favor, digite um passatempo.")
                    continue
                consultar_generico(
                    pedido,
                    prompt_template,
                    chat,
                    historico,
                    historico_labels=("Pedido", "Resposta")
                )
            except (EOFError, KeyboardInterrupt):
                logar_erro("Entrada interrompida pelo usuário.")
                print("\n🏁 Módulo encerrado.\n")
                break


In [26]:
# BLOCO 26

# Módulo Feedback
def feedback_gemini(chat, use_gui=False, output_widget=None):
    if use_gui:
        local_output = widgets.Output()

        def on_submit(input_widget):
            with local_output:
                clear_output()
                comentario = input_widget.value.strip()
                if not comentario:
                    print("⚠️ Por favor, digite um comentário.")
                    return
                try:
                    with open("feedback_cuidar.txt", "a", encoding="utf-8") as f:
                        f.write(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] {comentario}\n")
                    print("\n✅ Feedback registrado com sucesso!\n")
                    if input("💾 Deseja baixar o feedback? (s/n): ").strip().lower() == "s":
                        files.download("feedback_cuidar.txt")
                    input_widget.value = ""
                except Exception as e:
                    logar_erro(f"Erro ao salvar feedback: {e}")

        def on_exit(b):
            with local_output:
                clear_output()
                print("🏁 Módulo Feedback encerrado. Obrigado!")
            if output_widget:
                with output_widget:
                    clear_output()
                    print("🏁 Módulo Feedback encerrado. Obrigado!")

        interface, input_widget = montar_interface(
            pergunta_label="Feedback:",
            on_submit=on_submit,
            on_exit=on_exit
        )
        display(widgets.VBox([interface, local_output]))

    else:
        print("\n📝 Módulo de Feedback - Digite 'fim' para encerrar.\n")
        while True:
            try:
                comentario = input("❓ O que achou do Cuidar.AI? Sugira melhorias ou deixe seu comentário: ").strip()
                if comentario.lower() == "fim":
                    print("\n🏁 Módulo encerrado. Obrigado pelo feedback!\n")
                    break
                if not comentario:
                    print("⚠️ Por favor, digite um comentário.")
                    continue
                try:
                    with open("feedback_cuidar.txt", "a", encoding="utf-8") as f:
                        f.write(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] {comentario}\n")
                    print("\n✅ Feedback registrado com sucesso!\n")
                    if input("💾 Deseja baixar o feedback? (s/n): ").strip().lower() == "s":
                        files.download("feedback_cuidar.txt")
                except Exception as e:
                    logar_erro(f"Erro ao salvar feedback: {e}")
            except (EOFError, KeyboardInterrupt):
                logar_erro("Entrada interrompida pelo usuário.")
                print("\n🏁 Módulo encerrado. Obrigado pelo feedback!\n")
                break


In [27]:
# BLOCO 27

from io import StringIO  # ✅ Importação necessária para simular input

# Bloco de teste
def testar_modulos_cuidar_ai():
    print("\n🧪 Iniciando testes para módulos do Cuidar.AI...")

    try:
        import google.generativeai as genai
        from google.colab import userdata
        api_key = userdata.get('GOOGLE_API_KEY')
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel('gemini-1.5-flash')
        chat = model.start_chat(history=[])
    except Exception as e:
        erro_msg = f"❌ Erro ao configurar Gemini para teste: {e}"
        print(erro_msg)
        logar_erro(erro_msg)
        return

    def executar_teste(nome, entradas, funcao):
        print(f"\n📋 {nome}")
        original_input = sys.stdin
        try:
            sys.stdin = StringIO(entradas)
            funcao(chat, use_gui=False)
            print(f"✅ {nome} concluído.")
        except Exception as e:
            erro_msg = f"❌ {nome} falhou: {e}"
            print(erro_msg)
            logar_erro(erro_msg)
        finally:
            sys.stdin = original_input

    executar_teste("Teste 1: Saúde Preventiva (modo texto)", "Como prevenir diabetes?\ns\nn\nfim\n", saude_preventiva_gemini)
    executar_teste("Teste 2: Serviços Públicos (modo texto)", "Como tirar CPF?\ns\nn\nfim\n", servicos_publicos_gemini)
    executar_teste("Teste 3: Tradução Cidadã (modo texto)", "Art. 5º da Constituição\ns\nn\nfim\n", traducao_cidada_gemini)
    executar_teste("Teste 4: Tradução para Outras Línguas (modo texto)", "Olá, mundo!\ninglês\ns\nn\nfim\n", traducao_linguas_gemini)
    executar_teste("Teste 5: Tradução Cultural Brasileira (modo texto)", "Tá na pindaíba\ns\nn\nfim\n", traducao_cultural_gemini)
    executar_teste("Teste 6: Emergência Cidadã (modo texto)", "Acidente de carro\ns\nn\nfim\n", emergencia_cidada_gemini)
    executar_teste("Teste 7: Segurança Doméstica (modo texto)", "Vazamento de gás\ns\nn\nfim\n", emergencia_domestica_gemini)
    executar_teste("Teste 8: Emergência Infantil (modo texto)", "Bebê engasgado\ns\nn\nfim\n", emergencia_infantil_gemini)
    executar_teste("Teste 9: Apoio Emocional (modo texto)", "Estou ansioso\ns\nn\nfim\n", apoio_psicologico_gemini)
    executar_teste("Teste 10: Passa Tempo (modo texto)", "Piada\ns\nn\nfim\n", passatempo_gemini)
    executar_teste("Teste 11: Feedback (modo texto)", "Ótimo projeto!\nfim\n", feedback_gemini)

    # Teste 12: Modo Gráfico (simulado)
    print("\n🖥️ Teste 12: Modo Gráfico (simulado)")
    try:
        output_widget = widgets.Output()
        display(output_widget)
        with output_widget:
            print("ℹ️ Teste manual: Execute cada módulo com use_gui=True e interaja com os widgets.")
            print("Exemplo: servicos_publicos_gemini(chat, use_gui=True, output_widget=output_widget)")
        print("✅ Teste 12 concluído: Interface gráfica inicializada.")
    except Exception as e:
        erro_msg = f"❌ Teste 12 falhou: {e}"
        print(erro_msg)
        logar_erro(erro_msg)

    print("\n🧪 Testes finalizados.")

# Executar testes (descomente para rodar)
# testar_modulos_cuidar_ai()


# EATAPA 02.1 - Módulos Multiagentes

## 🌐 Módulo Especial: Cadeia de Agentes - Monitor de Tendências com Google Gemini

In [36]:
# BLOCO 28 - Planejador de Cidadania Digital

import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files
import google.generativeai as genai
import time

def cidadania_digital_gemini(chat, use_gui=False, output_widget=None):
    def processar_etapa(prompt, entrada, etapa_nome):
        try:
            response = chat.send_message(prompt.format(entrada=entrada))
            return response.text.strip()
        except Exception as e:
            logar_erro(f"Erro na etapa {etapa_nome}: {e}")
            return f"❌ Erro na etapa {etapa_nome}: {e}"

    def salvar_conteudo(conteudo, filename="conteudo_cidadania.txt"):
        with open(filename, "w", encoding="utf-8") as f:
            f.write(conteudo)
        files.download(filename)

    if use_gui:
        tema_input = widgets.Text(description="Tema:", placeholder="Ex.: saneamento")
        buscar_button = widgets.Button(description="Buscar", button_style="primary")
        planejar_button = widgets.Button(description="Planejar", button_style="info", disabled=True)
        redigir_button = widgets.Button(description="Redigir", button_style="success", disabled=True)
        salvar_button = widgets.Button(description="Salvar", button_style="warning", disabled=True)
        output = output_widget or widgets.Output()
        estado = {"tema": "", "busca": "", "plano": "", "conteudo": ""}

        def on_buscar_clicked(b):
            with output:
                clear_output()
                estado["tema"] = tema_input.value.strip()
                if not estado["tema"]:
                    print("⚠️ Por favor, insira um tema.")
                    return
                print(f"🔎 Agente de Busca: Buscando na web sobre {estado['tema']}...")
                prompt_busca = "Aja como se tivesse acessado a web agora e resuma informações recentes sobre '{entrada}' em até 100 palavras, como se estivesse consultando fontes confiáveis. Pergunte ao usuário se quer focar em algum aspecto específico."
                estado["busca"] = processar_etapa(prompt_busca, estado["tema"], "Busca")
                print(estado["busca"])
                buscar_button.disabled = True
                planejar_button.disabled = False
                tema_input.disabled = True

        def on_planejar_clicked(b):
            with output:
                clear_output()
                foco = input("Digite o aspecto específico ou pressione Enter para continuar: ").strip()
                print(f"🧠 Agente de Planejamento: Planejando conteúdo sobre {estado['tema']}...")
                prompt_planejamento = "Com base em '{entrada}', planeje um conteúdo educativo sobre {tema} (foco: {foco}) em até 100 palavras. Sugira um formato (ex.: guia, vídeo) e peça ao usuário para escolher."
                estado["plano"] = processar_etapa(prompt_planejamento.format(entrada=estado["busca"], tema=estado["tema"], foco=foco or "geral"), "", "Planejamento")
                print(estado["plano"])
                planejar_button.disabled = True
                redigir_button.disabled = False

        def on_redigir_clicked(b):
            with output:
                clear_output()
                formato = input("Digite o formato desejado (ex.: texto, vídeo) ou pressione Enter: ").strip()
                print(f"✍️ Agente de Redação: Redigindo conteúdo sobre {estado['tema']}...")
                prompt_redacao = "Com base em '{entrada}', redija um conteúdo educativo no formato {formato} sobre {tema} em até 200 palavras, usando linguagem inclusiva e cidadã."
                estado["conteudo"] = processar_etapa(prompt_redacao.format(entrada=estado["plano"], formato=formato or "texto", tema=estado["tema"]), "", "Redação")
                print(f"🎉 Conteúdo Final:\n{estado['conteudo']}")
                redigir_button.disabled = True
                salvar_button.disabled = False

        def on_salvar_clicked(b):
            with output:
                clear_output()
                salvar_conteudo(estado["conteudo"])
                print("🏁 Módulo encerrado.")
                salvar_button.disabled = True

        buscar_button.on_click(on_buscar_clicked)
        planejar_button.on_click(on_planejar_clicked)
        redigir_button.on_click(on_redigir_clicked)
        salvar_button.on_click(on_salvar_clicked)
        display(widgets.VBox([tema_input, buscar_button, planejar_button, redigir_button, salvar_button, output]))

    else:
        print("\n🛍️ Módulo Planejador de Cidadania Digital\n")
        tema = input("❓ Qual tema social você quer explorar? ").strip()
        if not tema:
            print("⚠️ Tema inválido. Encerrando.")
            return

        # Etapa 1: Busca
        print(f"🔎 Agente de Busca: Buscando na web sobre {tema}...")
        prompt_busca = "Aja como se tivesse acessado a web agora e resuma informações recentes sobre '{entrada}' em até 100 palavras, como se estivesse consultando fontes confiáveis. Pergunte ao usuário se quer focar em algum aspecto específico."
        busca = processar_etapa(prompt_busca, tema, "Busca")
        print(busca)
        foco = input("Digite o aspecto específico ou pressione Enter para continuar: ").strip()

        # Etapa 2: Planejamento
        print(f"🧠 Agente de Planejamento: Planejando conteúdo sobre {tema}...")
        prompt_planejamento = "Com base em '{entrada}', planeje um conteúdo educativo sobre {tema} (foco: {foco}) em até 100 palavras. Sugira um formato (ex.: guia, vídeo) e peça ao usuário para escolher."
        plano = processar_etapa(prompt_planejamento.format(entrada=busca, tema=tema, foco=foco or "geral"), "", "Planejamento")
        print(plano)
        formato = input("Digite o formato desejado (ex.: texto, vídeo) ou pressione Enter: ").strip()

        # Etapa 3: Redação
        print(f"✍️ Agente de Redação: Redigindo conteúdo sobre {tema}...")
        prompt_redacao = "Com base em '{entrada}', redija um conteúdo educativo no formato {formato} sobre {tema} em até 200 palavras, usando linguagem inclusiva e cidadã."
        conteudo = processar_etapa(prompt_redacao.format(entrada=plano, formato=formato or "texto", tema=tema), "", "Redação")
        print(f"\n🎉 Conteúdo Final Gerado pelo Cuidar.AI:\n\n════════════════════════════════════════\n{conteudo}\n════════════════════════════════════════")

        # Salvar
        salvar = input("💾 Deseja salvar este conteúdo? (s/n): ").strip().lower()
        if salvar == "s":
            salvar_conteudo(conteudo)
        print("\n🏁 Módulo Planejador de Cidadania Digital encerrado.")

In [37]:
# BLOCO 29 - Combate à Desinformação

import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files
import google.generativeai as genai
import time

def combate_desinformacao_gemini(chat, use_gui=False, output_widget=None):
    def processar_etapa(prompt, entrada, etapa_nome):
        try:
            response = chat.send_message(prompt.format(entrada=entrada))
            return response.text.strip()
        except Exception as e:
            logar_erro(f"Erro na etapa {etapa_nome}: {e}")
            return f"❌ Erro na etapa {etapa_nome}: {e}"

    def salvar_conteudo(conteudo, filename="alerta_desinformacao.txt"):
        with open(filename, "w", encoding="utf-8") as f:
            f.write(conteudo)
        files.download(filename)

    if use_gui:
        noticia_input = widgets.Textarea(description="Notícia:", placeholder="Cole a notícia ou URL")
        verificar_button = widgets.Button(description="Verificar", button_style="primary")
        avaliar_button = widgets.Button(description="Avaliar", button_style="info", disabled=True)
        redigir_button = widgets.Button(description="Redigir", button_style="success", disabled=True)
        salvar_button = widgets.Button(description="Salvar", button_style="warning", disabled=True)
        output = output_widget or widgets.Output()
        estado = {"noticia": "", "verificacao": "", "avaliacao": "", "alerta": ""}

        def on_verificar_clicked(b):
            with output:
                clear_output()
                estado["noticia"] = noticia_input.value.strip()
                if not estado["noticia"]:
                    print("⚠️ Por favor, insira uma notícia ou URL.")
                    return
                print(f"🔍 Agente de Verificação: Verificação de fatos sobre a notícia...")
                prompt_verificacao = "Aja como se tivesse acessado a web agora e verifique a veracidade da notícia ou URL '{entrada}' em até 100 palavras, como se estivesse consultando fontes confiáveis. Pergunte se o usuário quer detalhes adicionais."
                estado["verificacao"] = processar_etapa(prompt_verificacao, estado["noticia"], "Verificação")
                print(estado["verificacao"])
                verificar_button.disabled = True
                avaliar_button.disabled = False
                noticia_input.disabled = True

        def on_avaliar_clicked(b):
            with output:
                clear_output()
                detalhes = input("Digite detalhes adicionais ou pressione Enter: ").strip()
                print(f"🧠 Agente de Avaliação: Avaliando impacto social...")
                prompt_avaliacao = "Com base em '{entrada}', avalie o impacto social da desinformação sobre '{noticia}' (detalhes: {detalhes}) em até 100 palavras. Sugira um tipo de alerta (ex.: texto, vídeo)."
                estado["avaliacao"] = processar_etapa(prompt_avaliacao.format(entrada=estado["verificacao"], noticia=estado["noticia"], detalhes=detalhes or "nenhum"), "", "Avaliação")
                print(estado["avaliacao"])
                avaliar_button.disabled = True
                redigir_button.disabled = False

        def on_redigir_clicked(b):
            with output:
                clear_output()
                tipo_alerta = input("Digite o tipo de alerta (ex.: texto, vídeo) ou pressione Enter: ").strip()
                print(f"✍️ Agente de Redação: Redigindo alerta educativo...")
                prompt_redacao = "Com base em '{entrada}', redija um alerta educativo no formato {formato} sobre '{noticia}' em até 200 palavras, com linguagem cidadã."
                estado["alerta"] = processar_etapa(prompt_redacao.format(entrada=estado["avaliacao"], formato=tipo_alerta or "texto", noticia=estado["noticia"]), "", "Redação")
                print(f"📢 Alerta Final:\n{estado['alerta']}")
                redigir_button.disabled = True
                salvar_button.disabled = False

        def on_salvar_clicked(b):
            with output:
                clear_output()
                salvar_conteudo(estado["alerta"])
                print("🏁 Módulo encerrado.")
                salvar_button.disabled = True

        verificar_button.on_click(on_verificar_clicked)
        avaliar_button.on_click(on_avaliar_clicked)
        redigir_button.on_click(on_redigir_clicked)
        salvar_button.on_click(on_salvar_clicked)
        display(widgets.VBox([noticia_input, verificar_button, avaliar_button, redigir_button, salvar_button, output]))

    else:
        print("\n🚨 Módulo Combate à Desinformação - Cuidar.AI\n")
        noticia = input("❓ Copie e cole a notícia ou URL que deseja verificar: ").strip()
        if not noticia:
            print("⚠️ Notícia inválida. Encerrando.")
            return

        # Etapa 1: Verificação
        print(f"🔍 Agente de Verificação: Verificação de fatos...")
        prompt_verificacao = "Aja como se tivesse acessado a web agora e verifique a veracidade da notícia ou URL '{entrada}' em até 100 palavras, como se estivesse consultando fontes confiáveis. Pergunte se o usuário quer detalhes adicionais."
        verificacao = processar_etapa(prompt_verificacao, noticia, "Verificação")
        print(verificacao)
        detalhes = input("Digite detalhes adicionais ou pressione Enter: ").strip()

        # Etapa 2: Avaliação
        print(f"🧠 Agente de Avaliação: Avaliando impacto social...")
        prompt_avaliacao = "Com base em '{entrada}', avalie o impacto social da desinformação sobre '{noticia}' (detalhes: {detalhes}) em até 100 palavras. Sugira um tipo de alerta (ex.: texto, vídeo)."
        avaliacao = processar_etapa(prompt_avaliacao.format(entrada=verificacao, noticia=noticia, detalhes=detalhes or "nenhum"), "", "Avaliação")
        print(avaliacao)
        tipo_alerta = input("Digite o tipo de alerta (ex.: texto, vídeo) ou pressione Enter: ").strip()

        # Etapa 3: Redação
        print(f"✍️ Agente de Redação: Redigindo alerta educativo...")
        prompt_redacao = "Com base em '{entrada}', redija um alerta educativo no formato {formato} sobre '{noticia}' em até 200 palavras, com linguagem cidadã."
        alerta = processar_etapa(prompt_redacao.format(entrada=avaliacao, formato=tipo_alerta or "texto", noticia=noticia), "", "Redação")
        print(f"\n📢 Alerta final gerado pelo Cuidar.AI:\n\n════════════════════════════════════════\n{alerta}\n════════════════════════════════════════")

        # Salvar
        salvar = input("💾 Deseja salvar este alerta? (s/n): ").strip().lower()
        if salvar == "s":
            salvar_conteudo(alerta)
        print("\n🏁 Módulo Combate à Desinformação encerrado.")

In [39]:
# BLOCO 30 - Módulo de Testes para os Agentes Multi-etapa

# Objetivo: Automatizar os testes para os módulos 28 e 29
# Avalia se os agentes conseguem executar suas tarefas com sucesso, sem falhas de execução

from io import StringIO
import sys

# 🔧 Função de teste automatizado para os novos módulos multiagente
def testar_modulos_multiagente():
    print("\n📊 Iniciando testes para os módulos multiagente do Cuidar.AI (BLOCOS 28 e 29)...")

    try:
        import google.generativeai as genai
        from google.colab import userdata
        api_key = userdata.get("GOOGLE_API_KEY")
        genai.configure(api_key=api_key)
    except Exception as e:
        erro_msg = f"❌ Erro ao configurar Gemini para testes: {e}"
        print(erro_msg)
        logar_erro(erro_msg)
        return

    def simular_input(entradas, funcao):
        original_stdin = sys.stdin
        try:
            sys.stdin = StringIO(entradas)
            funcao(None)
            print("✅ Teste concluído com sucesso!\n")
        except Exception as e:
            erro_msg = f"❌ Erro durante o teste: {e}"
            print(erro_msg)
            logar_erro(erro_msg)
        finally:
            sys.stdin = original_stdin

    print("\n🔹 Teste Módulo 28 - Planejador de Cidadania Digital")
    simular_input("educação ambiental\ns\n", cidadania_digital_gemini)

    print("\n🔹 Teste Módulo 29 - Combate à Desinformação")
    simular_input("Está circulando que água com limão cura câncer. Isso é verdade?\ns\n", combate_desinformacao_gemini)

    print("\n🌟 Testes finalizados para os módulos multiagente.")

# Para executar diretamente, descomente a linha abaixo:
# testar_modulos_multiagente()


# ETAPA 03 - Menu de Opções do Cuidar.AI

In [40]:
# BLOCO 31 - Inicialização com Interface Gráfica (GUI) do Cuidar.AI

import ipywidgets as widgets
from IPython.display import display, clear_output
import google.generativeai as genai
from google.colab import userdata
import time
from google.api_core.exceptions import DeadlineExceeded

# Função para tentar inicializar o modelo Gemini
def try_initialize_model(model_name):
    for attempt in range(3):
        try:
            print(f"\nTentativa {attempt + 1}/3: Inicializando {model_name}...")
            available_models = [m.name for m in genai.list_models()]
            if f"models/{model_name}" not in available_models:
                error_msg = f"Modelo {model_name} não disponível."
                logar_erro(error_msg)
                print(f"\n❌ {error_msg}\nTentando o modelo alternativo...")
                model_name = "gemini-1.5-pro" if model_name == "gemini-1.5-flash" else "gemini-1.5-flash"
                continue
            model = genai.GenerativeModel(model_name, request_options={"timeout": 10})
            chat = model.start_chat(history=[])
            print(f"✅ {model_name} inicializado com sucesso.")
            return chat
        except DeadlineExceeded:
            logar_erro(f"Timeout ao inicializar {model_name}")
            time.sleep(2)
        except Exception as e:
            logar_erro(f"Erro ao inicializar {model_name}: {e}")
            time.sleep(2)
    print(f"\n❌ Todas as tentativas falharam para {model_name}.")
    return None

# GUI para escolher e inicializar modelo Gemini
def initialize_model_gui():
    # Validar chave API antes de criar a GUI
    api_key = userdata.get("GOOGLE_API_KEY")
    if not api_key:
        logar_erro("Chave API não encontrada em userdata.")
        print("❌ Erro: Chave API não configurada. Adicione 'GOOGLE_API_KEY' em userdata.")
        return

    try:
        genai.configure(api_key=api_key)
    except Exception as e:
        logar_erro(f"Erro ao configurar API: {e}")
        print(f"❌ Erro na configuração da API: {e}")
        return

    # Listar modelos disponíveis dinamicamente
    try:
        available_models = [m.name.split("/")[-1] for m in genai.list_models() if "gemini" in m.name]
        modelos = {f"Gemini {m.replace('gemini-', '')}": m for m in available_models}
        if not modelos:
            logar_erro("Nenhum modelo Gemini disponível.")
            print("❌ Nenhum modelo Gemini disponível. Verifique sua conexão ou chave API.")
            return
    except Exception as e:
        logar_erro(f"Erro ao listar modelos: {e}")
        print(f"❌ Erro ao listar modelos: {e}")
        return

    modelo_dropdown = widgets.Dropdown(
        options=["Selecione um modelo"] + list(modelos.keys()),
        description="Modelo:",
        style={'description_width': 'initial'}
    )

    iniciar_button = widgets.Button(description="Iniciar", button_style="success")
    output = widgets.Output()

    def on_iniciar_clicked(b):
        with output:
            clear_output()
            modelo_selecionado = modelo_dropdown.value
            if modelo_selecionado == "Selecione um modelo":
                print("⚠️ Por favor, selecione um modelo válido.")
                return

            model_name = modelos.get(modelo_selecionado)
            print(f"⏳ Carregando modelo {model_name}, por favor aguarde...")
            chat = try_initialize_model(model_name)

            clear_output()
            if chat:
                print(f"✅ Modelo {model_name} pronto!\n")
                criar_menu(chat, use_gui=True, output_widget=output)
            else:
                print("❌ Falha ao inicializar o modelo. Verifique sua chave de API ou conexão.")

    iniciar_button.on_click(on_iniciar_clicked)

    display(widgets.VBox([
        widgets.HTML("<h3>🤖 Selecione o modelo de IA do Cuidar.AI</h3>"),
        modelo_dropdown,
        iniciar_button,
        output
    ]))

In [41]:
# BLOCO 32 - Removido (Execução movida para o Bloco 34)

# NOTA: Este bloco foi removido, pois sua lógica de execução principal (tela de boas-vindas,
# escolha entre GUI e texto) é redundante com o Bloco 34. O Bloco 34 será o único ponto de
# entrada principal, chamando initialize_model_gui (Bloco 31) e criar_menu (Bloco 33).
# Se necessário, funções auxiliares podem ser definidas aqui no futuro.

pass

In [42]:
# BLOCO 33 - Menu Principal do Cuidar.AI

import ipywidgets as widgets
from IPython.display import display, clear_output
import google.generativeai as genai
import time

# Funções e módulos importados (assumindo que estão definidas na primeira parte)
# Substitua por: from cuidar_ai import logar_erro, responder_com_voz, consultar_generico, ...
from __main__ import (
    logar_erro, responder_com_voz, consultar_generico,
    saude_preventiva_gemini, servicos_publicos_gemini, traducao_cidada_gemini,
    traducao_cultural_gemini, traducao_linguas_gemini, emergencia_cidada_gemini,
    emergencia_domestica_gemini, emergencia_infantil_gemini, apoio_psicologico_gemini,
    passatempo_gemini, feedback_gemini, cidadania_digital_gemini, combate_desinformacao_gemini
)

# Interface do menu principal
def criar_menu(chat, use_gui=False, output_widget=None):
    # Validar sessão de chat
    if not hasattr(chat, 'send_message'):
        print("\n❌ Erro: Sessão de chat inválida. Reinicie o programa.")
        logar_erro("Sessão de chat inválida em criar_menu.")
        return
    try:
        # Teste simples para verificar se a sessão está ativa
        chat.send_message("teste", stream=False)
    except Exception as e:
        print("\n❌ Erro: Sessão de chat não funcional. Reinicie o programa.")
        logar_erro(f"Sessão de chat não funcional: {e}")
        return

    modulos = [
        "Saúde Preventiva",
        "Serviços Públicos",
        "Tradução Cidadã",
        "Tradução para Outras Línguas",
        "Tradução Cultural Brasileira",
        "Emergência Cidadã",
        "Segurança Doméstica",
        "Emergência Infantil",
        "Apoio Emocional e Prevenção ao Suicídio",
        "Passa Tempo",
        "Planejador de Cidadania Digital",
        "Combate à Desinformação",
        "Feedback"
    ]

    if use_gui:
        modulo_dropdown = widgets.Dropdown(
            options=["Selecione um módulo"] + modulos + ["Sair"],
            description="Módulo:",
            style={'description_width': 'initial'}
        )
        executar_button = widgets.Button(description="Executar", button_style="success")
        reiniciar_button = widgets.Button(description="Reiniciar", button_style="info")
        voltar_button = widgets.Button(description="Voltar", button_style="warning")
        output = output_widget or widgets.Output()

        def on_executar_clicked(b):
            with output:
                clear_output()
                modulo = modulo_dropdown.value
                if modulo == "Selecione um módulo":
                    print("⚠️ Por favor, selecione um módulo.")
                    return
                elif modulo == "Sair":
                    modulo_dropdown.disabled = True
                    executar_button.disabled = True
                    reiniciar_button.disabled = False
                    voltar_button.disabled = False
                    print("✅ Cuidar.AI pausado. Clique em 'Voltar' ou 'Reiniciar'.")
                    return
                try:
                    print(f"⏳ Carregando módulo {modulo}, por favor aguarde...")
                    if modulo == "Saúde Preventiva":
                        saude_preventiva_gemini(chat, use_gui=True, output_widget=output)
                    elif modulo == "Serviços Públicos":
                        servicos_publicos_gemini(chat, use_gui=True, output_widget=output)
                    elif modulo == "Tradução Cidadã":
                        traducao_cidada_gemini(chat, use_gui=True, output_widget=output)
                    elif modulo == "Tradução para Outras Línguas":
                        traducao_linguas_gemini(chat, use_gui=True, output_widget=output)
                    elif modulo == "Tradução Cultural Brasileira":
                        traducao_cultural_gemini(chat, use_gui=True, output_widget=output)
                    elif modulo == "Emergência Cidadã":
                        emergencia_cidada_gemini(chat, use_gui=True, output_widget=output)
                    elif modulo == "Segurança Doméstica":
                        emergencia_domestica_gemini(chat, use_gui=True, output_widget=output)
                    elif modulo == "Emergência Infantil":
                        emergencia_infantil_gemini(chat, use_gui=True, output_widget=output)
                    elif modulo == "Apoio Emocional e Prevenção ao Suicídio":
                        apoio_psicologico_gemini(chat, use_gui=True, output_widget=output)
                    elif modulo == "Passa Tempo":
                        passatempo_gemini(chat, use_gui=True, output_widget=output)
                    elif modulo == "Planejador de Cidadania Digital":
                        print("⚠️ Nota: Este módulo não suporta interface gráfica.")
                        cidadania_digital_gemini(chat)
                    elif modulo == "Combate à Desinformação":
                        print("⚠️ Nota: Este módulo não suporta interface gráfica.")
                        combate_desinformacao_gemini(chat)
                    elif modulo == "Feedback":
                        feedback_gemini(chat, use_gui=True, output_widget=output)
                    clear_output()
                    print(f"✅ Módulo {modulo} executado. Selecione outro módulo.")
                except Exception as e:
                    error_msg = f"Erro ao executar {modulo}: {e}"
                    logar_erro(error_msg)
                    clear_output()
                    print(f"❌ {error_msg}")

        def on_reiniciar_clicked(b):
            with output:
                clear_output()
                modulo_dropdown.disabled = False
                executar_button.disabled = False
                reiniciar_button.disabled = True
                voltar_button.disabled = True
                modulo_dropdown.value = "Selecione um módulo"
                print("🔄 Menu reiniciado. Selecione um módulo para continuar.")

        def on_voltar_clicked(b):
            with output:
                clear_output()
                modulo_dropdown.disabled = False
                executar_button.disabled = False
                reiniciar_button.disabled = True
                voltar_button.disabled = True
                modulo_dropdown.value = "Selecione um módulo"
                print("🔙 Menu principal restaurado.")

        executar_button.on_click(on_executar_clicked)
        reiniciar_button.on_click(on_reiniciar_clicked)
        voltar_button.on_click(on_voltar_clicked)
        reiniciar_button.disabled = True
        voltar_button.disabled = True
        display(widgets.VBox([modulo_dropdown, executar_button, reiniciar_button, voltar_button, output]))

    else:
        while True:
            print("\n══════════════════════")
            print("📋 MENU CUIDAR.AI")
            print("══════════════════════")
            print("== Assistência Pessoal e Cidadã ==")
            print("[1] Saúde Preventiva")
            print("[2] Serviços Públicos")
            print("[3] Tradução Cidadã")
            print("[4] Tradução para Outras Línguas")
            print("[5] Tradução Cultural Brasileira")
            print("\n== Emergência e Segurança ==")
            print("[6] Emergência Cidadã")
            print("[7] Segurança Doméstica")
            print("[8] Emergência Infantil")
            print("[9] Apoio Emocional e Prevenção ao Suicídio")
            print("\n== Lazer e Acessibilidade ==")
            print("[10] Passa Tempo")
            print("\n== Cidadania e Educação Digital ==")
            print("[11] Planejador de Cidadania Digital")
            print("[12] Combate à Desinformação")
            print("\n== Feedback ==")
            print("[13] Feedback")
            print("\n[0] Sair")
            print("══════════════════════\n")

            try:
                opcao = input("Escolha uma opção: ").strip()
                if opcao == "0":
                    print("👋 Cuidar.AI encerrado.")
                    break
                elif opcao == "1":
                    saude_preventiva_gemini(chat, use_gui=False)
                elif opcao == "2":
                    servicos_publicos_gemini(chat, use_gui=False)
                elif opcao == "3":
                    traducao_cidada_gemini(chat, use_gui=False)
                elif opcao == "4":
                    traducao_linguas_gemini(chat, use_gui=False)
                elif opcao == "5":
                    traducao_cultural_gemini(chat, use_gui=False)
                elif opcao == "6":
                    emergencia_cidada_gemini(chat, use_gui=False)
                elif opcao == "7":
                    emergencia_domestica_gemini(chat, use_gui=False)
                elif opcao == "8":
                    emergencia_infantil_gemini(chat, use_gui=False)
                elif opcao == "9":
                    apoio_psicologico_gemini(chat, use_gui=False)
                elif opcao == "10":
                    passatempo_gemini(chat, use_gui=False)
                elif opcao == "11":
                    cidadania_digital_gemini(chat)
                elif opcao == "12":
                    combate_desinformacao_gemini(chat)
                elif opcao == "13":
                    feedback_gemini(chat, use_gui=False)
                else:
                    print("\n⚠️ Opção inválida. Digite um número de 0 a 13.\n")
            except (EOFError, KeyboardInterrupt):
                print("\n👋 Cuidar.AI encerrado.")
                break

In [43]:
# BLOCO 34 - Inicialização e Execução Principal do Cuidar.AI

from IPython.display import clear_output
import google.generativeai as genai
from google.colab import userdata
import time
from io import StringIO
import sys

# Funções importadas (assumindo que estão definidas em outros blocos)
# Substitua por: from cuidar_ai import initialize_model_gui, criar_menu, logar_erro, cidadania_digital_gemini, combate_desinformacao_gemini
from __main__ import initialize_model_gui, criar_menu, logar_erro, cidadania_digital_gemini, combate_desinformacao_gemini

# Versão do sistema
versao_atual = "v1.0.0"

# Função para tentar inicializar o modelo Gemini
def try_initialize_model(model_name):
    for attempt in range(3):
        try:
            print(f"\nTentativa {attempt + 1}/3: Inicializando {model_name}...")
            available_models = [m.name for m in genai.list_models()]
            if f"models/{model_name}" not in available_models:
                error_msg = f"Modelo {model_name} não disponível."
                logar_erro(error_msg)
                print(f"\n❌ {error_msg}\nTentando o modelo alternativo...")
                model_name = "gemini-1.5-pro" if model_name == "gemini-1.5-flash" else "gemini-1.5-flash"
                continue
            model = genai.GenerativeModel(model_name)
            chat = model.start_chat(history=[])
            print(f"✅ {model_name} inicializado com sucesso.")
            return chat
        except Exception as e:
            logar_erro(f"Erro ao inicializar {model_name}: {e}")
            print(f"❌ Erro: {e}")
            time.sleep(2)
    print(f"\n❌ Todas as tentativas falharam para {model_name}.")
    return None

# Função para inicializar modelo via interface textual
def initialize_model_text():
    modelos = {"1": "gemini-1.5-flash", "2": "gemini-1.5-pro"}
    while True:
        print("\nEscolha o modelo de IA:")
        print("[1] Gemini 1.5 Flash (rápido)")
        print("[2] Gemini 1.5 Pro (detalhado)")
        modelo_opcao = input("Digite 1 ou 2: ").strip()
        if modelo_opcao in modelos:
            return try_initialize_model(modelos[modelo_opcao])
        print("⚠️ Opção inválida. Tente novamente.")

# Função para testar módulos multiagentes
def testar_modulos_agentes():
    print("\n🧪 Testando módulos Planejador de Cidadania Digital e Combate à Desinformação...")
    try:
        genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))
        model = genai.GenerativeModel("gemini-1.5-flash")
        chat = model.start_chat(history=[])
        print("\n✅ Inicialização de modelo para teste concluída.")

        original_stdin = sys.stdin
        try:
            print("\n📋 Teste 1: Planejador de Cidadania Digital")
            sys.stdin = StringIO("educação ambiental\nn\n")
            cidadania_digital_gemini(chat)
            print("✅ Módulo Planejador de Cidadania Digital executado com sucesso.")

            print("\n📋 Teste 2: Combate à Desinformação")
            sys.stdin = StringIO("Água com limão cura câncer\nn\n")
            combate_desinformacao_gemini(chat)
            print("✅ Módulo Combate à Desinformação executado com sucesso.")
        finally:
            sys.stdin = original_stdin
    except Exception as e:
        logar_erro(f"Erro geral nos testes de agentes: {e}")
        print(f"❌ Erro nos testes: {e}")

# Execução principal
clear_output()
print(f"🎉 Bem-vindo ao Cuidar.AI - Versão {versao_atual}")
print("Escolha o tipo de interface:")
print("[1] Botões (interface gráfica)")
print("[2] Menu (interface de texto)")

try:
    # Validar chave API
    api_key = userdata.get("GOOGLE_API_KEY")
    if not api_key:
        logar_erro("Chave API não encontrada em userdata.")
        print("❌ Erro: Chave API não configurada. Adicione 'GOOGLE_API_KEY' em userdata.")
        raise ValueError("Chave API ausente")

    # Configuração global da API Gemini
    genai.configure(api_key=api_key)

    escolha = input("Digite 1 ou 2: ").strip()
    if escolha not in ["1", "2"]:
        print("⚠️ Opção inválida. Escolha 1 ou 2.")
        raise ValueError("Entrada inválida")

    if escolha == "1":
        initialize_model_gui()
    else:
        chat = initialize_model_text()
        if chat:
            criar_menu(chat, use_gui=False)
        else:
            print("\n❌ Falha na inicialização do modelo. Encerrando.")
            raise ValueError("Falha na inicialização do modelo")

except Exception as e:
    logar_erro(f"Erro crítico na execução do Cuidar.AI: {e}")
    print(f"❌ Erro inesperado: {e}")

# Para executar testes manualmente:
# testar_modulos_agentes()

🎉 Bem-vindo ao Cuidar.AI - Versão v1.0.0
Escolha o tipo de interface:
[1] Botões (interface gráfica)
[2] Menu (interface de texto)
Digite 1 ou 2: 2

Escolha o modelo de IA:
[1] Gemini 1.5 Flash (rápido)
[2] Gemini 1.5 Pro (detalhado)
Digite 1 ou 2: 1

Tentativa 1/3: Inicializando gemini-1.5-flash...
✅ gemini-1.5-flash inicializado com sucesso.

══════════════════════
📋 MENU CUIDAR.AI
══════════════════════
== Assistência Pessoal e Cidadã ==
[1] Saúde Preventiva
[2] Serviços Públicos
[3] Tradução Cidadã
[4] Tradução para Outras Línguas
[5] Tradução Cultural Brasileira

== Emergência e Segurança ==
[6] Emergência Cidadã
[7] Segurança Doméstica
[8] Emergência Infantil
[9] Apoio Emocional e Prevenção ao Suicídio

== Lazer e Acessibilidade ==
[10] Passa Tempo

== Cidadania e Educação Digital ==
[11] Planejador de Cidadania Digital
[12] Combate à Desinformação

== Feedback ==
[13] Feedback

[0] Sair
══════════════════════

Escolha uma opção: 11

🛍️ Módulo Planejador de Cidadania Digital

❓ Q